# now do some ML using the web scraped data

In [236]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [237]:
from player_data_expansion import get_all_expanded_player_fight_data

In [238]:
# AWS integration getting file data
df_init = pd.read_csv("fight_data.csv").fillna(pd.NA)
df_init.head()

,Date,fighter1,fighter1_result,method_or_round,fighter2,Venue,Undercard fights,link,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage
0,"Sunday, 4 May 2025",Naoya Inoue,Winner,"TKO, Round 8",Ramon Cardenas,"T-Mobile Arena, Las Vegas, United States",Rafael Espinoza Beat Edward Vazquez (TKO 7) \...,https://box.live/fights/inoue-vs-cardenas/,29,0,0,26,0,1,164,168.0,0,0
1,"Saturday, 3 May 2025",Canelo Alvarez,Winner,"UD, Round 12",William Scull,"The Venue, Riyadh, Saudi Arabia",Jaime Munguia Beat Bruno Surace (UD 12) \nMar...,https://box.live/fights/canelo-vs-scull/,62,2,2,23,0,0,175,182.0,0,0
2,"Saturday, 3 May 2025",Jukiya Iimura,Winner,"UD, Round 12",Esneth Domingo,"Korakuen Hall, Tokyo, Japan",<NA>,https://box.live/fights/iimura-vs-domingo/,8,0,1,21,0,2,164,160.0,0,0
3,"Friday, 2 May 2025",Ryan Garcia,Loser,"UD, Round 12",Rolando ‘Rolly’ Romero,"Times Square, New York, NY, United States",Devin Haney Beat Jose Ramirez (UD 12) \nTeofi...,https://box.live/fights/garcia-vs-romero/,24,0,1,16,0,2,178,173.0,83,81
4,"Thursday, 1 May 2025",Kevin Lerena,Winner,"TKO, Round 3",Serhiy Radchenko,"Sunbet Arena, Pretoria, South Africa",<NA>,https://box.live/fights/lerena-vs-radchenko/,30,0,3,11,0,7,185,186.0,47,45


In [239]:
# ADD BASIC DATA TO SEE IF IT WORKS
df_init = pd.concat([pd.read_csv("fight_data_backup_basic.csv"), df_init], axis=0)
df_init.drop_duplicates(inplace=True)

### start cleaning

In [240]:
df_essentials = df_init.drop(columns=['link','Venue','Date','Undercard fights','fighter1','fighter2'])
df_essentials.head(2)

,fighter1_result,method_or_round,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage
0,Winner,"RTD, Round 3",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Winner,"UD, Round 12",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [241]:
df_staged_cleaning = df_essentials.copy()
df_staged_cleaning.head(2)

,fighter1_result,method_or_round,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage
0,Winner,"RTD, Round 3",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Winner,"UD, Round 12",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [242]:
df_staged_cleaning['method_or_round'] = df_staged_cleaning['method_or_round'].str.split(' ')

In [243]:
last_rounds = [new_list[2] for new_list in df_staged_cleaning['method_or_round']][:]
df_staged_cleaning['round'] = last_rounds

df_staged_cleaning.head()

,fighter1_result,method_or_round,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage,round
0,Winner,"[RTD,, Round, 3]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1,Winner,"[UD,, Round, 12]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
2,Winner,"[TKO,, Round, 2]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,Winner,"[UD,, Round, 12]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
4,Winner,"[TKO,, Round, 7]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7


In [244]:
df_staged_cleaning = df_staged_cleaning.drop(columns='method_or_round')
df_staged_cleaning.head()

,fighter1_result,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage,round
0,Winner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1,Winner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
2,Winner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,Winner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
4,Winner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7


### feature engineering swapped values + add to current values

In [245]:

df_staged_swapping = df_staged_cleaning.copy()

fighter2_wins_temp = df_staged_swapping['fighter2_record_wins'].copy()
fighter2_draws_temp = df_staged_swapping['fighter2_record_draws'].copy()
fighter2_losses_temp = df_staged_swapping['fighter2_record_losses'].copy()
fighter2_height_cm_temp = df_staged_swapping['fighter2_height_cm'].copy()
fighter2_ko_percentage_temp = df_staged_swapping['fighter2_ko_percentage'].copy()

df_staged_swapping['fighter2_record_wins'] = df_staged_swapping['fighter1_record_wins']
df_staged_swapping['fighter2_record_draws'] = df_staged_swapping['fighter1_record_draws']
df_staged_swapping['fighter2_record_losses'] = df_staged_swapping['fighter1_record_losses']
df_staged_swapping['fighter2_height_cm'] = df_staged_swapping['fighter1_height_cm']
df_staged_swapping['fighter2_ko_percentage'] = df_staged_swapping['fighter1_ko_percentage']

df_staged_swapping['fighter1_record_wins'] = fighter2_wins_temp
df_staged_swapping['fighter1_record_draws'] = fighter2_draws_temp
df_staged_swapping['fighter1_record_losses'] = fighter2_losses_temp
df_staged_swapping['fighter1_height_cm'] = fighter2_height_cm_temp
df_staged_swapping['fighter1_ko_percentage'] = fighter2_ko_percentage_temp

def reverse_results(rowval) : 
    if rowval == 'Winner' :
        return 'Loser'
    elif rowval == 'Loser' :
        return 'Winner'
    else :
        return 'Draw'
    
df_staged_swapping['fighter1_result'] = df_staged_swapping['fighter1_result'].apply(reverse_results)


concatenate the matrices

In [246]:
df_encoded = pd.concat([df_staged_cleaning, df_staged_swapping], axis=0)

In [247]:
df_encoded

,fighter1_result,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage,round
0,Winner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1,Winner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
2,Winner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,Winner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
4,Winner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
5,Draw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10
6,Winner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
7,Winner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10
8,Winner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
9,Winner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6


could also feature engineer the ratio between fighters, such as fighter2/fighter1 for height

### encoding

In [248]:
# commented out so that the concatenation of swapped fighters is correctly set
# df_encoded = df_staged_cleaning.copy()

In [249]:
# WINNER = 0, LOSER = 1, DRAW = 2
def custom_encoder(value) :
    if value == 'Winner' :
        return 0
    elif value == 'Loser' :
        return 1
    else : return 2
df_encoded['fighter1_result'] = df_encoded['fighter1_result'].apply(custom_encoder)

In [250]:
df_encoded.shape

(48, 12)

In [251]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48 entries, 0 to 11
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   fighter1_result         48 non-null     int64  
 1   fighter1_record_wins    24 non-null     float64
 2   fighter1_record_draws   24 non-null     float64
 3   fighter1_record_losses  24 non-null     float64
 4   fighter2_record_wins    24 non-null     float64
 5   fighter2_record_draws   24 non-null     float64
 6   fighter2_record_losses  24 non-null     float64
 7   fighter1_height_cm      23 non-null     float64
 8   fighter2_height_cm      23 non-null     float64
 9   fighter1_ko_percentage  24 non-null     float64
 10  fighter2_ko_percentage  24 non-null     float64
 11  round                   48 non-null     object 
dtypes: float64(10), int64(1), object(1)
memory usage: 4.9+ KB


In [252]:
df_encoded = df_encoded.infer_objects()
df_encoded['fighter1_record_wins'] = df_encoded['fighter1_record_wins'].astype('Int64')
df_encoded['fighter1_record_draws'] = df_encoded['fighter1_record_draws'].astype('Int64')
df_encoded['fighter1_record_losses'] = df_encoded['fighter1_record_losses'].astype('Int64')
df_encoded['fighter2_record_wins'] = df_encoded['fighter2_record_wins'].astype('Int64')
df_encoded['fighter2_record_draws'] = df_encoded['fighter2_record_draws'].astype('Int64')
df_encoded['fighter2_record_losses'] = df_encoded['fighter1_record_losses'].astype('Int64')
df_encoded['fighter2_height_cm'] = df_encoded['fighter2_height_cm'].astype('Int64')
df_encoded['fighter1_ko_percentage'] = df_encoded['fighter1_ko_percentage'].astype('Int64')
df_encoded['fighter2_ko_percentage'] = df_encoded['fighter2_ko_percentage'].astype('Int64')
df_encoded['round'] = df_encoded['round'].astype('Int64')

In [253]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48 entries, 0 to 11
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   fighter1_result         48 non-null     int64  
 1   fighter1_record_wins    24 non-null     Int64  
 2   fighter1_record_draws   24 non-null     Int64  
 3   fighter1_record_losses  24 non-null     Int64  
 4   fighter2_record_wins    24 non-null     Int64  
 5   fighter2_record_draws   24 non-null     Int64  
 6   fighter2_record_losses  24 non-null     Int64  
 7   fighter1_height_cm      23 non-null     float64
 8   fighter2_height_cm      23 non-null     Int64  
 9   fighter1_ko_percentage  24 non-null     Int64  
 10  fighter2_ko_percentage  24 non-null     Int64  
 11  round                   48 non-null     Int64  
dtypes: Int64(10), float64(1), int64(1)
memory usage: 5.3 KB


In [254]:
df_encoded.describe()

,fighter1_result,fighter1_record_wins,fighter1_record_draws,fighter1_record_losses,fighter2_record_wins,fighter2_record_draws,fighter2_record_losses,fighter1_height_cm,fighter2_height_cm,fighter1_ko_percentage,fighter2_ko_percentage,round
count,48.00000,24.0,24.0,24.0,24.0,24.0,24.0,23.000000,23.0,24.0,24.0,48.0
mean,0.62500,23.5,0.333333,2.083333,23.5,0.333333,2.083333,174.913043,174.913043,26.625,26.625,7.833333
std,0.63998,11.481554,0.56466,2.205067,11.481554,0.56466,2.205067,8.974272,8.974272,33.657886,33.657886,3.639811
min,0.00000,8.0,0.0,0.0,8.0,0.0,0.0,160.000000,160.0,0.0,0.0,2.0
25%,0.00000,15.75,0.0,0.0,15.75,0.0,0.0,167.000000,167.0,0.0,0.0,5.25
50%,1.00000,23.0,0.0,1.5,23.0,0.0,1.5,175.000000,175.0,0.0,0.0,7.5
75%,1.00000,26.75,1.0,3.25,26.75,1.0,3.25,182.500000,182.5,59.5,59.5,12.0
max,2.00000,62.0,2.0,7.0,62.0,2.0,7.0,191.000000,191.0,83.0,83.0,12.0


### tt split

In [255]:
from sklearn.model_selection import train_test_split

In [256]:
X = df_encoded.drop(columns=['fighter1_result'])
y = df_encoded['fighter1_result']
y.unique()

array([0, 2, 1], dtype=int64)

In [257]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=66) # tweak to maker sure that y values contain wins, losses, and draws. ts = .08 and rs =  66
print(y_test.unique())
print(y_train.unique())

[0 1 2]
[1 2 0]


## TRAIN MODEL - XGBoost First

In [294]:
from xgboost import XGBClassifier

In [295]:
xgbc = XGBClassifier(
    objective='multi:softmax',
    random_state=42,
    num_class=3,
    verbosity=1
)

In [296]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [297]:
y_train = encoder.fit_transform(y_train)

In [298]:
# compute sample weight based on class frequency
class_weights = dict(enumerate(len(y_train) / (len(np.unique(y_train)) * np.bincount(y_train))))
sample_weight = np.array([class_weights[label] for label in y_train])
sample_weight

array([0.66666667, 0.66666667, 0.66666667, 0.66666667, 0.66666667,
       4.22222222, 0.66666667, 0.79166667, 0.79166667, 0.66666667,
       0.66666667, 0.66666667, 0.79166667, 0.79166667, 0.66666667,
       0.79166667, 0.66666667, 0.79166667, 0.79166667, 0.66666667,
       0.79166667, 0.79166667, 0.66666667, 0.66666667, 0.66666667,
       0.79166667, 0.66666667, 4.22222222, 0.66666667, 0.79166667,
       0.79166667, 4.22222222, 0.79166667, 0.79166667, 0.66666667,
       0.66666667, 0.79166667, 0.79166667])

In [299]:
# # no class weighting
# xgbc.fit(X_train, y_train)

# class weighting
xgbc.fit(X_train, y_train, sample_weight=sample_weight)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_class=3, ...)

In [300]:
xgbc_preds = xgbc.predict(X_test)
xgbc_preds = encoder.inverse_transform(xgbc_preds)

In [301]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [302]:
print(accuracy_score(y_test, xgbc_preds))

0.7


In [303]:
print(confusion_matrix(y_test, xgbc_preds))

[[4 2 0]
 [1 2 0]
 [0 0 1]]


In [304]:
print(classification_report(y_test, xgbc_preds))

              precision    recall  f1-score   support

           0       0.80      0.67      0.73         6
           1       0.50      0.67      0.57         3
           2       1.00      1.00      1.00         1

    accuracy                           0.70        10
   macro avg       0.77      0.78      0.77        10
weighted avg       0.73      0.70      0.71        10



In [305]:
xgbc_preds

array([1, 1, 0, 2, 1, 0, 0, 0, 0, 1], dtype=int64)

### grid search cross validated + XGBoost Classifier - same rseult as initial one

In [306]:
from sklearn.model_selection import GridSearchCV

In [307]:
eval_set = [(X, y)] # validation set for early stopping

In [308]:
xgbc_model2 = XGBClassifier(
    objective='multi:softmax',
    random_state=42,
    num_class=3,
    verbosity=1,
    eval_metric="mlogloss"
)

In [309]:
param_grid = {
    "subsample":[0.75, 1],
     "colsample_bytree":[0.75, 1],
     "max_depth":[2, 6, 10],
     "min_child_weight":[1, 2,3,4,5],
     "learning_rate":[1, 0.1, 0.01, .001],
}

In [310]:
xgbc_grid = GridSearchCV(
    xgbc_model2,
    param_grid,
    cv=3,
    scoring="roc_auc"
)

In [311]:
xgbc_grid.fit(X_train,y_train,eval_set=eval_set,verbose=1)

[0]	validation_0-mlogloss:0.96453
[1]	validation_0-mlogloss:0.80944
[2]	validation_0-mlogloss:0.77228
[3]	validation_0-mlogloss:0.74481
[4]	validation_0-mlogloss:0.76554
[5]	validation_0-mlogloss:0.74140
[6]	validation_0-mlogloss:0.74174
[7]	validation_0-mlogloss:0.74265
[8]	validation_0-mlogloss:0.75926
[9]	validation_0-mlogloss:0.74050
[10]	validation_0-mlogloss:0.76246
[11]	validation_0-mlogloss:0.79272
[12]	validation_0-mlogloss:0.77046
[13]	validation_0-mlogloss:0.75684
[14]	validation_0-mlogloss:0.76252
[15]	validation_0-mlogloss:0.80361
[16]	validation_0-mlogloss:0.78380
[17]	validation_0-mlogloss:0.79451
[18]	validation_0-mlogloss:0.76415
[19]	validation_0-mlogloss:0.75598
[20]	validation_0-mlogloss:0.74843
[21]	validation_0-mlogloss:0.82766
[22]	validation_0-mlogloss:0.83130
[23]	validation_0-mlogloss:0.80780
[24]	validation_0-mlogloss:0.81422
[25]	validation_0-mlogloss:0.78534
[26]	validation_0-mlogloss:0.78314
[27]	validation_0-mlogloss:0.77631
[28]	validation_0-mlogloss:0.7

[51]	validation_0-mlogloss:0.76787
[52]	validation_0-mlogloss:0.74857
[53]	validation_0-mlogloss:0.72733
[54]	validation_0-mlogloss:0.75669
[55]	validation_0-mlogloss:0.73156
[56]	validation_0-mlogloss:0.77825
[57]	validation_0-mlogloss:0.75999
[58]	validation_0-mlogloss:0.74645
[59]	validation_0-mlogloss:0.75178
[60]	validation_0-mlogloss:0.75431
[61]	validation_0-mlogloss:0.74657
[62]	validation_0-mlogloss:0.73858
[63]	validation_0-mlogloss:0.73768
[64]	validation_0-mlogloss:0.75634
[65]	validation_0-mlogloss:0.77617
[66]	validation_0-mlogloss:0.77236
[67]	validation_0-mlogloss:0.79383
[68]	validation_0-mlogloss:0.74986
[69]	validation_0-mlogloss:0.74976
[70]	validation_0-mlogloss:0.76407
[71]	validation_0-mlogloss:0.78238
[72]	validation_0-mlogloss:0.80013
[73]	validation_0-mlogloss:0.76516
[74]	validation_0-mlogloss:0.76771
[75]	validation_0-mlogloss:0.80994
[76]	validation_0-mlogloss:0.76717
[77]	validation_0-mlogloss:0.76660
[78]	validation_0-mlogloss:0.77195
[79]	validation_0-ml

c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 376, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

[1]	validation_0-mlogloss:0.95416
[2]	validation_0-mlogloss:0.92773
[3]	validation_0-mlogloss:0.85003
[4]	validation_0-mlogloss:0.87332
[5]	validation_0-mlogloss:0.83694
[6]	validation_0-mlogloss:0.84086
[7]	validation_0-mlogloss:0.84182
[8]	validation_0-mlogloss:0.84603
[9]	validation_0-mlogloss:0.81031
[10]	validation_0-mlogloss:0.80233
[11]	validation_0-mlogloss:0.79892
[12]	validation_0-mlogloss:0.79727
[13]	validation_0-mlogloss:0.78866
[14]	validation_0-mlogloss:0.78893
[15]	validation_0-mlogloss:0.77009
[16]	validation_0-mlogloss:0.77676
[17]	validation_0-mlogloss:0.77145
[18]	validation_0-mlogloss:0.79464
[19]	validation_0-mlogloss:0.80088
[20]	validation_0-mlogloss:0.77297
[21]	validation_0-mlogloss:0.76367
[22]	validation_0-mlogloss:0.77774
[23]	validation_0-mlogloss:0.78277
[24]	validation_0-mlogloss:0.78200
[25]	validation_0-mlogloss:0.79563
[26]	validation_0-mlogloss:0.79733
[27]	validation_0-mlogloss:0.81354
[28]	validation_0-mlogloss:0.81286
[29]	validation_0-mlogloss:0.

c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 376, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

...

[91]	validation_0-mlogloss:0.87239
[92]	validation_0-mlogloss:0.87231
[93]	validation_0-mlogloss:0.87263
[94]	validation_0-mlogloss:0.87287
[95]	validation_0-mlogloss:0.87282
[96]	validation_0-mlogloss:0.87320
[97]	validation_0-mlogloss:0.87342
[98]	validation_0-mlogloss:0.87337
[99]	validation_0-mlogloss:0.87375
[0]	validation_0-mlogloss:0.97586
[1]	validation_0-mlogloss:0.94007
[2]	validation_0-mlogloss:0.93518
[3]	validation_0-mlogloss:0.92119
[4]	validation_0-mlogloss:0.94207
[5]	validation_0-mlogloss:0.92692
[6]	validation_0-mlogloss:0.93380
[7]	validation_0-mlogloss:0.92467
[8]	validation_0-mlogloss:0.90553
[9]	validation_0-mlogloss:0.88775
[10]	validation_0-mlogloss:0.91578
[11]	validation_0-mlogloss:0.90165
[12]	validation_0-mlogloss:0.87509
[13]	validation_0-mlogloss:0.87276
[14]	validation_0-mlogloss:0.87682
[15]	validation_0-mlogloss:0.85523
[16]	validation_0-mlogloss:0.85943
[17]	validation_0-mlogloss:0.87260
[18]	validation_0-mlogloss:0.87187
[19]	validation_0-mlogloss:0.8

c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 376, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

[98]	validation_0-mlogloss:0.76189
[99]	validation_0-mlogloss:0.76270
[0]	validation_0-mlogloss:0.96845
[1]	validation_0-mlogloss:0.89687
[2]	validation_0-mlogloss:0.90169
[3]	validation_0-mlogloss:0.88601
[4]	validation_0-mlogloss:0.89098
[5]	validation_0-mlogloss:0.84052
[6]	validation_0-mlogloss:0.85496
[7]	validation_0-mlogloss:0.82923
[8]	validation_0-mlogloss:0.82984
[9]	validation_0-mlogloss:0.83730
[10]	validation_0-mlogloss:0.85225
[11]	validation_0-mlogloss:0.81311
[12]	validation_0-mlogloss:0.83123
[13]	validation_0-mlogloss:0.83128
[14]	validation_0-mlogloss:0.82472
[15]	validation_0-mlogloss:0.81576
[16]	validation_0-mlogloss:0.80291
[17]	validation_0-mlogloss:0.80334
[18]	validation_0-mlogloss:0.82138
[19]	validation_0-mlogloss:0.86367
[20]	validation_0-mlogloss:0.84647
[21]	validation_0-mlogloss:0.81513
[22]	validation_0-mlogloss:0.83012
[23]	validation_0-mlogloss:0.83716
[24]	validation_0-mlogloss:0.82197
[25]	validation_0-mlogloss:0.81325
[26]	validation_0-mlogloss:0.8

c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 376, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

KeyboardInterrupt: 

In [ ]:
xgbc_grid.best_params_

{'colsample_bytree': 0.75,
 'learning_rate': 1,
 'max_depth': 2,
 'min_child_weight': 1,
 'subsample': 0.75}

In [ ]:
xgbc_cv_best = XGBClassifier(
    colsample_bytree = 0.75,
    learning_rate = 0.1,
    max_depth = 2,
    min_child_weight = 1,
    subsample = 0.75,
    objective='multi:softmax',
    random_state=42,
    num_class=3,
    verbosity=1,
    eval_metric="mlogloss"
)

In [ ]:
xgbc_cv_best.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.75, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=3, ...)

In [ ]:
xgbc_cv_best_preds = xgbc_cv_best.predict(X_test)
xgbc_cv_best_preds = encoder.inverse_transform(xgbc_cv_best_preds)

In [ ]:
print(confusion_matrix(y_test, xgbc_cv_best_preds),"\n")
print(classification_report(y_test, xgbc_cv_best_preds))

[[4 2 0]
 [2 1 0]
 [0 1 0]] 

              precision    recall  f1-score   support

           0       0.67      0.67      0.67         6
           1       0.25      0.33      0.29         3
           2       0.00      0.00      0.00         1

    accuracy                           0.50        10
   macro avg       0.31      0.33      0.32        10
weighted avg       0.47      0.50      0.49        10



c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X_train.shape[1]

11

In [ ]:
rfc_1 = RandomForestClassifier(
    n_estimators=round(np.sqrt(X_train.shape[1])),
    random_state=42
    )

In [ ]:
rfc_1.fit(X_train,y_train)

RandomForestClassifier(n_estimators=3, random_state=42)

In [ ]:
rfc_1_preds = rfc_1.predict(X_test)

In [ ]:
rfc_1_preds = encoder.inverse_transform(rfc_1_preds)

In [ ]:
print(classification_report(y_test,rfc_1_preds))
print(confusion_matrix(y_test,rfc_1_preds))

              precision    recall  f1-score   support

           0       0.67      0.33      0.44         6
           1       0.29      0.67      0.40         3
           2       0.00      0.00      0.00         1

    accuracy                           0.40        10
   macro avg       0.32      0.33      0.28        10
weighted avg       0.49      0.40      0.39        10

[[2 4 0]
 [1 2 0]
 [0 1 0]]


c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Dylan\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
